In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('health_ai_mdav_demo.csv')
df.head()

,ID,Age,Sex,ZIP,SystolicBP,BMI,Diagnosis
0,1,27,F,53116,108,27.1,Hypertension
1,2,66,M,53118,123,25.2,Migraine
2,3,59,M,53118,126,22.8,Asthma
3,4,47,F,53116,108,24.9,Diabetes
4,5,46,F,53118,110,25.0,Migraine


In [3]:
df.info()

# Quasi-Identifiers (QIs): Age, ZIP Code, Sex
# We don't include SystolicBP as QIs because someone couldn't be traced by just looking at it.
# BMI changes significantly within a period of months to years and can be affected by a lot of other factors.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          40 non-null     int64  
 1   Age         40 non-null     int64  
 2   Sex         40 non-null     object 
 3   ZIP         40 non-null     int64  
 4   SystolicBP  40 non-null     int64  
 5   BMI         40 non-null     float64
 6   Diagnosis   40 non-null     object 
dtypes: float64(1), int64(4), object(2)
memory usage: 2.3+ KB


In [4]:
df.describe()

# No missing value in the data. 
# Assume that all fields are correctly inputed.

,ID,Age,ZIP,SystolicBP,BMI
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,20.500000,51.750000,53116.600000,114.600000,24.852500
std,11.690452,15.669258,1.150251,13.285427,2.720953
min,1.000000,25.000000,53115.000000,90.000000,20.100000
25%,10.750000,41.500000,53116.000000,103.750000,22.775000
50%,20.500000,52.500000,53117.000000,119.500000,24.750000
75%,30.250000,66.000000,53117.250000,124.000000,26.650000
max,40.000000,77.000000,53119.000000,143.000000,31.500000


In [5]:
# Step 0: censor the Sex data
df['Sex'] = "*"

In [6]:
# Step 1: extract numeric QIs for MDAV
qi_num = ['Age','ZIP']
qi_data = df[qi_num].copy()

# Normalize the data --> because the range are different.
# MDAV is based on distance, so it's better to have similar range
scaler = StandardScaler()
qi_normalized = scaler.fit_transform(qi_data)
qi_normalized_df = pd.DataFrame(qi_normalized, columns=qi_num, index=df.index)

qi_normalized_df.head()

,Age,ZIP
0,-1.599648,-0.528271
1,0.921010,1.232631
2,0.468584,1.232631
3,-0.307003,-0.528271
4,-0.371635,1.232631


In [7]:
# Step 3: measure how far two records using Euclidean
def euclidean_dist(point1,point2):
    return np.sqrt(np.sum((point1-point2) **2))

print(euclidean_dist(qi_normalized[0], qi_normalized[1]))

3.0748155073910377


In [8]:
# Step 4: apply MDAV --> microaggregation algorithm based on Euclidean
# MDAV creates clusters by repeatedly finding the "opposite ends" of your data, 
# ensuring groups are compact and information loss is minimized while achieving k-anonymity.

# Define k = 8 --> each group will have 8 members and there will be 5 groups in total
k = 8
n_records = len(qi_normalized)
unclustered = set(range(n_records)) # to track unclustered row
clusters = {}
cluster_id = 0 # start with 0

while len(unclustered) >= 2*k:
    print("="*50)
    print(f"Iteration: {cluster_id + 1}")
    print(f"Unclustered records: {len(unclustered)}")

    # Step a: calculate average vector of unclustered records
    unclustered_idx = list(unclustered)
    unclustered_data = qi_normalized[unclustered_idx]
    avg_vector = np.mean(unclustered_data, axis = 0)
    print(f"Avg. vector: {avg_vector}")

    # Step b: find max distant record from average
    dist_from_avg = [euclidean_dist(qi_normalized[i], avg_vector) for i in unclustered_idx]
    max_dist_idx = unclustered_idx[np.argmax(dist_from_avg)]
    print(f"Max dist idx: {max_dist_idx} | dist: {max(dist_from_avg):.3f}")

    # Step c: form first cluster - add k-1 nearest neighbors
    distances_from_most_distant = [(i, euclidean_dist(qi_normalized[i], qi_normalized[max_dist_idx])) for i in unclustered_idx]
    # Sort by distance
    distances_from_most_distant.sort(key=lambda x: x[1])
    cluster_1 = [idx for idx, dist in distances_from_most_distant[:k]]

    clusters[f"Cluster_{cluster_id}"] = cluster_1
    print(f"Cluster_{cluster_id} formed with records: {cluster_1}")
    
    # Remove from unclustered
    for idx in cluster_1:
        unclustered.remove(idx)
    cluster_id += 1
    
    # Check if we have enough records for another pair
    if len(unclustered) < 2 * k:
        break
        
    # Step d: find record furthest from first cluster's centroid
    cluster_1_centroid = np.mean(qi_normalized[cluster_1], axis=0)
    unclustered_idx = list(unclustered)
    distances_from_cluster1 = [euclidean_dist(qi_normalized[i], cluster_1_centroid) for i in unclustered_idx]
    opposite_idx = unclustered_idx[np.argmax(distances_from_cluster1)]
    print(f"Opposite record: {opposite_idx} (distance from Cluster_{cluster_id-1}: {max(distances_from_cluster1):.3f})")
    
    # Step e: form second cluster
    distances_from_opposite = [(i, euclidean_dist(qi_normalized[i], qi_normalized[opposite_idx])) for i in unclustered_idx]
    distances_from_opposite.sort(key=lambda x: x[1])
    cluster_2 = [idx for idx, dist in distances_from_opposite[:k]]
    clusters[f"Cluster_{cluster_id}"] = cluster_2
    print(f"Cluster_{cluster_id} formed with records: {cluster_2}")
    
    # Remove from unclustered
    for idx in cluster_2:
        unclustered.remove(idx)
    cluster_id += 1

# Step f: handle remaining records
if len(unclustered) > 0:
    remaining = list(unclustered)
    print(f"\n--- Final cluster ---")
    print(f"Remaining {len(remaining)} records: {remaining}")
    clusters[f"Cluster_{cluster_id}"] = remaining

Iteration: 1
Unclustered records: 40
Avg. vector: [-2.22044605e-17  1.28123068e-12]
Max dist idx: 32 | dist: 2.650
Cluster_0 formed with records: [32, 4, 17, 22, 8, 38, 21, 26]
Opposite record: 11 (distance from Cluster_0: 3.357)
Cluster_1 formed with records: [11, 34, 18, 36, 27, 7, 28, 33]
Iteration: 3
Unclustered records: 24
Avg. vector: [0.03770214 0.13206764]
Max dist idx: 9 | dist: 2.248
Cluster_2 formed with records: [9, 0, 6, 35, 31, 19, 30, 3]
Opposite record: 23 (distance from Cluster_2: 3.275)
Cluster_3 formed with records: [23, 5, 1, 15, 12, 14, 37, 2]

--- Final cluster ---
Remaining 8 records: [10, 13, 16, 20, 24, 25, 29, 39]


In [9]:
# Cluster name and idx
clusters

{'Cluster_0': [32, 4, 17, 22, 8, 38, 21, 26],
 'Cluster_1': [11, 34, 18, 36, 27, 7, 28, 33],
 'Cluster_2': [9, 0, 6, 35, 31, 19, 30, 3],
 'Cluster_3': [23, 5, 1, 15, 12, 14, 37, 2],
 'Cluster_4': [10, 13, 16, 20, 24, 25, 29, 39]}

In [10]:
# Step 5: Anonymize the data

df_anonymized = df.copy()

# For each cluster, replace QI values with cluster means
for cluster_name, indices in clusters.items():
    # Calculate mean of ORIGINAL (not normalized) values
    cluster_means = qi_data.iloc[indices][qi_num].mean().astype(int)
    
    print(f"\n{cluster_name} means:")
    print(cluster_means)
    
    # Replace values with means
    for col in qi_num:
        df_anonymized.loc[indices, col] = cluster_means[col]


Cluster_0 means:
Age       36
ZIP    53117
dtype: int64

Cluster_1 means:
Age       65
ZIP    53115
dtype: int64

Cluster_2 means:
Age       35
ZIP    53115
dtype: int64

Cluster_3 means:
Age       66
ZIP    53117
dtype: int64

Cluster_4 means:
Age       55
ZIP    53116
dtype: int64


In [11]:
# Output: df anonymized

df_anonymized.to_csv('health_ai_anonymized.csv', index=False)
df_anonymized.head()

,ID,Age,Sex,ZIP,SystolicBP,BMI,Diagnosis
0,1,35,*,53115,108,27.1,Hypertension
1,2,66,*,53117,123,25.2,Migraine
2,3,66,*,53117,126,22.8,Asthma
3,4,35,*,53115,108,24.9,Diabetes
4,5,36,*,53117,110,25.0,Migraine


In [12]:
# Output: df cluster assignment

cluster_assignments = []
for cluster_name, indices in clusters.items():
    for idx in indices:
        cluster_assignments.append({
            'RecordID': df.loc[idx, 'ID'],
            'OriginalIndex': idx,
            'ClusterID': cluster_name
        })

cluster_df = pd.DataFrame(cluster_assignments).sort_values('RecordID')
cluster_df.to_csv('cluster_assignments.csv', index=False)

cluster_df.head()

,RecordID,OriginalIndex,ClusterID
17,1,0,Cluster_2
26,2,1,Cluster_3
31,3,2,Cluster_3
23,4,3,Cluster_2
1,5,4,Cluster_0


In [13]:
# Output: quality metrics

# SSE
sse = 0
for cluster_name, idx in clusters.items():
    # Get cluster centroid in normalized space
    cluster_centroid_normalized = qi_normalized[idx].mean(axis=0)
    
    # Calculate squared errors
    for i in idx:
        error = qi_normalized[i] - cluster_centroid_normalized
        sse += np.sum(error ** 2)

print(f"Total SSE: {sse:.4f}")
print(f"Average SSE per record: {sse/len(df):.4f}")
# Higher = more information loss.

# Cluster Radius
cluster_rad = []
for cluster_name, idx in clusters.items():
    cluster_centroid_normalized = qi_normalized[idx].mean(axis=0)
    
    # Calculate average distance from centroid
    distances = [euclidean_dist(qi_normalized[i], cluster_centroid_normalized) for i in idx]
    avg_radius = np.mean(distances)
    cluster_rad.append(avg_radius)
    
    print(f"{cluster_name}: radius = {avg_radius:.4f}")

overall_avg_radius = np.mean(cluster_rad)
print(f"\n=== Average Cluster Radius ===")
print(f"Overall average: {overall_avg_radius:.4f}")
# Smaller = more homogeneous groups

Total SSE: 19.8987
Average SSE per record: 0.4975
Cluster_0: radius = 0.6576
Cluster_1: radius = 0.6647
Cluster_2: radius = 0.7306
Cluster_3: radius = 0.3499
Cluster_4: radius = 0.7434

=== Average Cluster Radius ===
Overall average: 0.6293
